In [37]:
%load_ext autoreload
%autoreload 2

import jax.numpy as jnp 

import geometry
import humidity

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
geometry.initialize_geometry()

(Array([0.  , 0.05, 0.14, 0.26, 0.42, 0.6 , 0.77, 0.9 , 1.  ], dtype=float32),
 Array([0.05      , 0.09      , 0.11999999, 0.16      , 0.18000004,
        0.16999996, 0.13      , 0.10000002], dtype=float32),
 Array([0.025     , 0.095     , 0.19999999, 0.33999997, 0.51      ,
        0.685     , 0.835     , 0.95      ], dtype=float32),
 Array([10.       ,  5.5555553,  4.166667 ,  3.125    ,  2.7777772,
         2.9411771,  3.846154 ,  4.999999 ], dtype=float32),
 Array([5.714286  , 1.5037595 , 0.7142858 , 0.42016813, 0.28011206,
        0.20855059, 0.17108642, 0.15037595], dtype=float32),
 Array([0.9988202 , 0.993582  , 0.98417646, 0.970643  , 0.9530382 ,
        0.9314361 , 0.9059272 , 0.87661856, 0.843633  , 0.80710906,
        0.76719975, 0.7240726 , 0.67790836, 0.62890077, 0.57725537,
        0.5231887 , 0.46692774, 0.40870813, 0.34877434, 0.28737777,
        0.22477564, 0.16123083, 0.09700976, 0.03238179],      dtype=float32, weak_type=True),
 Array([0.0485619 , 0.11311407, 0.17719

In [59]:
temp = jnp.array([[[273] * 96] * 48])
pressure = jnp.array([[[0.5] * 96] * 48])
sigma_levels = 4
qg = jnp.array([[[2] * 96] * 48])

qsat = humidity.get_qsat(temp, pressure, sigma_levels)

rh, qsat_new = humidity.spec_hum_to_rel_hum(temp, pressure, sigma_levels, qg)

rh, qsat_new

qa, qsat_new = humidity.rel_hum_to_spec_hum(temp, pressure, sigma_levels, rh)

float(jnp.take(qa, 0))

2.0